In [1]:
import pandas as pd
import numpy as np

import xgboost as xgb
from sklearn.model_selection import train_test_split, TimeSeriesSplit
from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score, mean_absolute_percentage_error

import matplotlib.pyplot as plt
import seaborn as sns

from datetime import datetime

import warnings
warnings.filterwarnings('ignore')

# load data

In [2]:
df_birth = pd.read_csv('./data/2_BirthsAndFertilityRatesAnnual.csv')
df_price = pd.read_csv('./data/COEBiddingResultsPrices.csv')

In [3]:
# clean data
df_price['bids_success'] = df_price['bids_success'].str.replace(',', '').astype(int)
df_price['bids_received'] = df_price['bids_received'].str.replace(',', '').astype(int)
df_price['bidding_no'] = df_price['bidding_no'].astype(str)

df_price['bid_success_rate'] = df_price['bids_success'] / df_price['bids_received']
df_price['bid_fulfil_rate'] = df_price['bids_success'] / df_price['quota']

# predictive model

In [55]:
class COECategoryPredictor:

    def __init__(self):
        self.models = {}
        self.scalers = {}
        self.label_encoder = LabelEncoder()
        self.feature_names = []
        
    def load_and_merge_data(self, coe_df, birth_df):

        coe_data = coe_df.copy()
        coe_data['year'] = coe_data['month'].str[:4].astype(int)
        coe_data['month_num'] = coe_data['month'].str[5:7].astype(int)
        
        birth_data = birth_df.copy()
        
        key_age_groups = ['25 - 29 Years', '30 - 34 Years', '35 - 39 Years', '40 - 44 Years']
        birth_features = {}
        
        for year in range(2010, 2025):
            year_str = str(year)
            if year_str in birth_data.columns:
                year_data = {}
                for idx, row in birth_data.iterrows():
                    age_group = row.iloc[0]
                    if age_group in key_age_groups:
                        clean_age = age_group.replace(' - ', '_').replace(' Years', '').replace(' ', '_')
                        year_data[f'birth_rate_{clean_age}'] = row[year_str]
                
                main_age_births = [year_data.get(f'birth_rate_{age}', 0) 
                                 for age in ['25_29', '30_34', '35_39']]
                year_data['birth_rate_main_age'] = sum([x for x in main_age_births if pd.notnull(x)])
                
                birth_features[year] = year_data

        for year, features in birth_features.items():
            for feature_name, value in features.items():
                coe_data.loc[coe_data['year'] == year, feature_name] = value

        return coe_data
    
    def create_features(self, df):
        
        df_feat = df.copy()
        df_feat = df_feat.sort_values(['vehicle_class', 'month', 'bidding_no'])
        
        # 1. demand
        df_feat['demand_supply_ratio'] = df_feat['bids_received'] / df_feat['quota']
        df_feat['excess_demand'] = df_feat['bids_received'] - df_feat['quota']
        df_feat['excess_demand_pct'] = df_feat['excess_demand'] / df_feat['quota']
        df_feat['success_rate'] = df_feat['bids_success'] / df_feat['bids_received']
        df_feat['unfulfilled_demand'] = df_feat['bids_received'] - df_feat['bids_success']
        
        # 2. time related
        df_feat['quarter'] = ((df_feat['month_num'] - 1) // 3 + 1)
        df_feat['is_year_end'] = (df_feat['month_num'] >= 11).astype(int)
        df_feat['is_mid_year'] = ((df_feat['month_num'] >= 6) & (df_feat['month_num'] <= 8)).astype(int)
        
        # seasonal
        df_feat['month_sin'] = np.sin(2 * np.pi * df_feat['month_num'] / 12)
        df_feat['month_cos'] = np.cos(2 * np.pi * df_feat['month_num'] / 12)
        
        # 3. historical
        for lag in [1, 2, 3, 6]:
            df_feat[f'premium_lag{lag}'] = df_feat.groupby('vehicle_class')['premium'].shift(lag)
            if lag <= 3:
                df_feat[f'quota_lag{lag}'] = df_feat.groupby('vehicle_class')['quota'].shift(lag)
                df_feat[f'demand_lag{lag}'] = df_feat.groupby('vehicle_class')['bids_received'].shift(lag)
        
        # 4. ma
        for window in [3, 6, 12]:
            df_feat[f'premium_ma{window}'] = df_feat.groupby('vehicle_class')['premium'].rolling(window).mean().reset_index(0, drop=True)
            df_feat[f'demand_supply_ratio_ma{window}'] = df_feat.groupby('vehicle_class')['demand_supply_ratio'].rolling(window).mean().reset_index(0, drop=True)
            if window <= 6:
                df_feat[f'quota_ma{window}'] = df_feat.groupby('vehicle_class')['quota'].rolling(window).mean().reset_index(0, drop=True)
        
        # 5. change ratio
        df_feat['premium_pct_change'] = df_feat.groupby('vehicle_class')['premium'].pct_change()
        df_feat['quota_change'] = df_feat.groupby('vehicle_class')['quota'].diff()
        df_feat['demand_change'] = df_feat.groupby('vehicle_class')['bids_received'].diff()
        
        # 6. volatility
        df_feat['premium_volatility_3m'] = df_feat.groupby('vehicle_class')['premium'].rolling(3).std().reset_index(0, drop=True)
        df_feat['premium_volatility_6m'] = df_feat.groupby('vehicle_class')['premium'].rolling(6).std().reset_index(0, drop=True)
        
        # 7. relative
        df_feat['premium_vs_ma3'] = df_feat['premium'] / df_feat['premium_ma3']
        df_feat['premium_vs_ma6'] = df_feat['premium'] / df_feat['premium_ma6']
        df_feat['ratio_vs_ma3'] = df_feat['demand_supply_ratio'] / df_feat['demand_supply_ratio_ma3']
        
        # 8. mean, std, min, max
        monthly_stats = df_feat.groupby('month').agg({
            'premium': ['mean', 'std', 'min', 'max'],
            'demand_supply_ratio': 'mean',
            'quota': 'sum'
        }).round(2)
        
        monthly_stats.columns = ['market_avg_premium', 'market_premium_std', 'market_min_premium', 
                               'market_max_premium', 'market_avg_ratio', 'total_quota']
        monthly_stats = monthly_stats.reset_index()
        
        df_feat = df_feat.merge(monthly_stats, on='month', how='left')
        
        df_feat['premium_vs_market'] = df_feat['premium'] / df_feat['market_avg_premium']
        df_feat['ratio_vs_market'] = df_feat['demand_supply_ratio'] / df_feat['market_avg_ratio']
        
        # 9. birth
        birth_cols = [col for col in df_feat.columns if col.startswith('birth_rate_')]
        
        for col in birth_cols:
            if col in df_feat.columns:
                df_feat[f'{col}_change'] = df_feat.groupby('vehicle_class')[col].pct_change()
                df_feat[f'{col}_ma3'] = df_feat.groupby('vehicle_class')[col].rolling(3).mean().reset_index(0, drop=True)
        
        df_feat['vehicle_class_encoded'] = self.label_encoder.fit_transform(df_feat['vehicle_class'])
        
        # clean data
        df_feat = df_feat.replace([np.inf, -np.inf], np.nan)
        
        return df_feat
    
    def prepare_modeling_data(self, df):
        feature_cols = [
            'quota', 'bids_received', 'bids_success', 'demand_supply_ratio', 'excess_demand_pct',
            'success_rate', 'unfulfilled_demand', 'vehicle_class_encoded',
            'year', 'month_num', 'quarter', 'is_year_end', 'is_mid_year', 'month_sin', 'month_cos',
            'premium_lag1', 'premium_lag2', 'premium_lag3', 'quota_lag1', 'demand_lag1',
            'premium_ma3', 'premium_ma6', 'demand_supply_ratio_ma3', 'quota_ma3',
            'premium_pct_change', 'quota_change', 'demand_change',
            'premium_volatility_3m', 'premium_volatility_6m',
            'premium_vs_ma3', 'premium_vs_ma6', 'premium_vs_market', 'ratio_vs_market',
            'market_avg_premium', 'market_premium_std', 'market_avg_ratio', 'total_quota',
            'birth_rate_main_age'
        ]
        
        available_features = [col for col in feature_cols if col in df.columns]
        self.feature_names = available_features
        
        # drop na
        df_clean = df[available_features + ['premium', 'vehicle_class', 'month']].dropna()
        
        print(f"Num features: {len(available_features)}")
        return df_clean
    
    def train_models(self, df):
        results = {}
        
        for category in df['vehicle_class'].unique():
            print(f"\train {category} XGBoost...")
            
            cat_data = df[df['vehicle_class'] == category].copy()
            cat_data = cat_data.sort_values('month')
            
            if len(cat_data) < 10:
                print(f"  {category} not enough records")
                continue
            
            X = cat_data[self.feature_names]
            y = cat_data['premium']
            
            split_point = int(len(cat_data) * 0.8)
            X_train, X_test = X[:split_point], X[split_point:]
            y_train, y_test = y[:split_point], y[split_point:]
            
            xgb_params = {
                'objective': 'reg:squarederror',
                'n_estimators': 200,
                'max_depth': 6,
                'learning_rate': 0.1,
                'subsample': 0.8,
                'colsample_bytree': 0.8,
                'random_state': 42,
                'n_jobs': -1
            }
            
            models = {
                'XGBoost_Default': xgb.XGBRegressor(**xgb_params),
                'XGBoost_DeepTree': xgb.XGBRegressor(**{**xgb_params, 'max_depth': 8, 'n_estimators': 150}),
                'XGBoost_HighLR': xgb.XGBRegressor(**{**xgb_params, 'learning_rate': 0.15, 'n_estimators': 150})
            }
            
            cat_results = {}
            
            for model_name, model in models.items():
                print(f"  train {model_name}...")
                
                model.fit(
                    X_train, y_train,
                    eval_set=[(X_train, y_train), (X_test, y_test)],
                    eval_metric=['mae', 'mape'],
                    early_stopping_rounds=20,
                    verbose=False
                )
                
                y_pred_train = model.predict(X_train)
                y_pred_test = model.predict(X_test)

                train_mae = mean_absolute_error(y_train, y_pred_train)
                test_mae = mean_absolute_error(y_test, y_pred_test)
                train_mape = mean_absolute_percentage_error(y_train, y_pred_train)
                test_mape = mean_absolute_percentage_error(y_test, y_pred_test)
                train_rmse = np.sqrt(mean_squared_error(y_train, y_pred_train))
                test_rmse = np.sqrt(mean_squared_error(y_test, y_pred_test))
                test_r2 = r2_score(y_test, y_pred_test)
                
                best_iteration = getattr(model, 'best_iteration', model.n_estimators)
                
                cat_results[model_name] = {
                    'model': model,
                    'train_mae': train_mae,
                    'test_mae': test_mae,
                    'train_mape': train_mape,
                    'test_mape': test_mape,
                    'train_rmse': train_rmse,
                    'test_rmse': test_rmse,
                    'test_r2': test_r2,
                    'best_iteration': best_iteration,
                    'y_test': y_test,
                    'y_pred': y_pred_test
                }
                
                print(f"    MAE=${test_mae:,.0f}, MAPE={test_mape:,.2f}, RMSE=${test_rmse:,.0f}, R²={test_r2:.3f}, Best iteration:{best_iteration}")

            best_model_name = min(cat_results.keys(), key=lambda x: cat_results[x]['test_mape'])
            best_result = cat_results[best_model_name]
            
            self.models[category] = best_result['model']
            
            results[category] = {
                'best_model': best_model_name,
                'all_results': cat_results,
                'sample_count': len(cat_data),
                'feature_importance': self._get_xgb_feature_importance(best_result['model'])
            }
            
            print(f"Best model: {best_model_name} (MAPE: {best_result['test_mape']:,.2f})")
        
        return results
    
    def _get_xgb_feature_importance(self, model):

        importance_dict = model.get_booster().get_score(importance_type='weight')
        importance_df = pd.DataFrame([
            {'feature': k, 'importance': v} 
            for k, v in importance_dict.items()
        ]).sort_values('importance', ascending=False)
        
        return importance_df
    
    def predict_category(self, category, features):
        if category not in self.models:
            raise ValueError(f"cannot find {category}")
        
        model = self.models[category]
        
        prediction = model.predict(features)
        
        return prediction
    
    def get_feature_importance(self, category):

        if category not in self.models:
            return None
        
        model = self.models[category]
        
        importance_types = ['weight', 'gain', 'cover']
        importance_results = {}
        
        for imp_type in importance_types:
            try:
                importance_dict = model.get_booster().get_score(importance_type=imp_type)
                importance_df = pd.DataFrame([
                    {'feature': k, 'importance': v} 
                    for k, v in importance_dict.items()
                ]).sort_values('importance', ascending=False)
                importance_results[imp_type] = importance_df
            except:
                pass
        
        return importance_results.get('weight', None)
    
    def plot_feature_importance(self, category, top_n=15, importance_type='weight'):
        if category not in self.models:
            print(f"Cannot find {category}")
            return
        
        model = self.models[category]
        
        try:
            importance_dict = model.get_booster().get_score(importance_type=importance_type)
            importance_df = pd.DataFrame([
                {'feature': k, 'importance': v} 
                for k, v in importance_dict.items()
            ]).sort_values('importance', ascending=False).head(top_n)
            
            plt.figure(figsize=(10, 8))
            plt.barh(range(len(importance_df)), importance_df['importance'])
            plt.yticks(range(len(importance_df)), importance_df['feature'])
            plt.xlabel(f'Feature Importance ({importance_type})')
            plt.title(f'{category} - XGBoost Feature Importance')
            plt.gca().invert_yaxis()
            plt.tight_layout()
            plt.show()
            
        except Exception as e:
            print(f"fail: {e}")
    
    def predict_with_confidence(self, category, features, n_estimators_range=(50, 200)):
        if category not in self.models:
            raise ValueError(f"Cannot find {category} ")
        
        model = self.models[category]
        predictions = []

        for n_trees in range(n_estimators_range[0], n_estimators_range[1], 25):
            temp_model = model
            temp_model.n_estimators = min(n_trees, model.n_estimators)
            pred = temp_model.predict(features.reshape(1, -1))[0]
            predictions.append(pred)
        
        mean_pred = np.mean(predictions)
        std_pred = np.std(predictions)
        
        return {
            'prediction': mean_pred,
            'confidence_interval': (mean_pred - 1.96*std_pred, mean_pred + 1.96*std_pred),
            'std_dev': std_pred
        }


In [56]:
predictor = COECategoryPredictor()
merged_data = predictor.load_and_merge_data(df_price, df_birth)
featured_data = predictor.create_features(merged_data)
model_data = predictor.prepare_modeling_data(featured_data)

Num features: 38


In [57]:
# train
results = predictor.train_models(model_data)

	rain Category A XGBoost...
  train XGBoost_Default...
    MAE=$5,304, MAPE=0.06, RMSE=$6,813, R²=0.606, Best iteration:103
  train XGBoost_DeepTree...
    MAE=$5,360, MAPE=0.06, RMSE=$6,895, R²=0.597, Best iteration:101
  train XGBoost_HighLR...
    MAE=$5,712, MAPE=0.06, RMSE=$7,711, R²=0.496, Best iteration:89
Best model: XGBoost_Default (MAPE: 0.06)
	rain Category B XGBoost...
  train XGBoost_Default...
    MAE=$18,431, MAPE=0.16, RMSE=$21,775, R²=-1.856, Best iteration:176
  train XGBoost_DeepTree...
    MAE=$18,004, MAPE=0.16, RMSE=$21,300, R²=-1.733, Best iteration:149
  train XGBoost_HighLR...
    MAE=$18,400, MAPE=0.16, RMSE=$21,777, R²=-1.856, Best iteration:135
Best model: XGBoost_DeepTree (MAPE: 0.16)
	rain Category C XGBoost...
  train XGBoost_Default...
    MAE=$5,106, MAPE=0.07, RMSE=$6,857, R²=0.613, Best iteration:156
  train XGBoost_DeepTree...
    MAE=$5,110, MAPE=0.07, RMSE=$6,840, R²=0.615, Best iteration:149
  train XGBoost_HighLR...
    MAE=$5,143, MAPE=0.07, RMS

In [58]:
# output
for category, result in results.items():
    print(f"\n{category}:")
    print(f"  Sample count: {result['sample_count']}")
    print(f"  best model: {result['best_model']}")
    
    if 'feature_importance' in result and result['feature_importance'] is not None:
        importance = result['feature_importance']
        print(f"  top 5 features:")
        for _, row in importance.head(5).iterrows():
            print(f"    {row['feature']}: {row['importance']:.1f}")

    best_model_results = result['all_results'][result['best_model']]
    print(f"  Metrics: MAE=${best_model_results['test_mae']:,.0f}, MAPE={best_model_results['test_mape']:,.2f}, R²={best_model_results['test_r2']:.3f}")
    print(f"  best interations: {best_model_results['best_iteration']}")




Category A:
  Sample count: 349
  best model: XGBoost_Default
  top 5 features:
    quota: 228.0
    premium_vs_market: 147.0
    premium_vs_ma3: 127.0
    premium_ma3: 124.0
    market_avg_premium: 120.0
  Metrics: MAE=$5,304, MAPE=0.06, R²=0.606
  best interations: 103

Category B:
  Sample count: 349
  best model: XGBoost_DeepTree
  top 5 features:
    quota: 656.0
    bids_received: 358.0
    premium_vs_market: 311.0
    demand_supply_ratio: 251.0
    premium_pct_change: 220.0
  Metrics: MAE=$18,004, MAPE=0.16, R²=-1.733
  best interations: 149

Category C:
  Sample count: 349
  best model: XGBoost_Default
  top 5 features:
    quota: 353.0
    bids_received: 221.0
    premium_ma3: 201.0
    premium_pct_change: 199.0
    premium_lag1: 196.0
  Metrics: MAE=$5,106, MAPE=0.07, R²=0.613
  best interations: 156

Category D:
  Sample count: 349
  best model: XGBoost_HighLR
  top 5 features:
    quota: 185.0
    premium_ma3: 147.0
    bids_received: 127.0
    premium_vs_ma6: 110.0
    pr

In [59]:
df_price.groupby('vehicle_class').premium.describe()

,count,mean,std,min,25%,50%,75%,max
vehicle_class,,,,,,,,
Category A,368.0,58380.500000,22291.714224,18502.0,40687.75,55299.5,74989.50,106000.0
Category B,368.0,69803.095109,27903.787629,19190.0,47000.75,65511.0,90625.75,150001.0
Category C,368.0,48048.399457,16788.484831,19001.0,32889.75,46945.5,58665.00,91101.0
Category D,368.0,5789.940217,3260.129143,852.0,2457.50,6101.0,8674.00,13189.0
Category E,368.0,71009.817935,28334.653375,19889.0,48009.50,65545.0,92391.75,158004.0


In [60]:
# feature top 10

for i in range(5):
    first_category = list(predictor.models.keys())[i]
    print(f"\n {first_category} feature importance:")
    importance = predictor.get_feature_importance(first_category)
    if importance is not None:
        print("Top 10 features:")
        for i, (_, row) in enumerate(importance.head(10).iterrows(), 1):
            print(f"  {i:2d}. {row['feature']}: {row['importance']:.1f}")


 Category A feature importance:
Top 10 features:
   1. quota: 228.0
   2. premium_vs_market: 147.0
   3. premium_vs_ma3: 127.0
   4. premium_ma3: 124.0
   5. market_avg_premium: 120.0
   6. premium_pct_change: 113.0
   7. premium_lag1: 113.0
   8. premium_vs_ma6: 108.0
   9. bids_received: 107.0
  10. premium_volatility_3m: 100.0

 Category B feature importance:
Top 10 features:
   1. quota: 656.0
   2. bids_received: 358.0
   3. premium_vs_market: 311.0
   4. demand_supply_ratio: 251.0
   5. premium_pct_change: 220.0
   6. premium_lag1: 213.0
   7. demand_change: 196.0
   8. quota_change: 196.0
   9. premium_volatility_3m: 192.0
  10. premium_lag3: 173.0

 Category C feature importance:
Top 10 features:
   1. quota: 353.0
   2. bids_received: 221.0
   3. premium_ma3: 201.0
   4. premium_pct_change: 199.0
   5. premium_lag1: 196.0
   6. demand_supply_ratio: 189.0
   7. premium_vs_ma3: 179.0
   8. premium_vs_ma6: 154.0
   9. premium_volatility_3m: 140.0
  10. demand_supply_ratio_ma3: 1

# quota suggestion

In [61]:
# 1. change quota values: Current ± 5%, ±10%, ±15%, ±20%
# 3. test in XGBoost model → Get predicted prices
# 4. Map quota change % to price change %

In [100]:
# para
ls_quota_features = ['quota',
#  'quota_lag1',
#  'quota_lag2',
#  'quota_lag3',
#  'quota_ma3',
#  'quota_ma6',
#  'quota_change',
#  'total_quota'
 ]

ls_ratio = [0.95, 1.05, 0.9, 1.1, 0.85, 1.15, 0.8, 1.2]
# ls_ratio = [0.1, .05]
ls_categories = list(predictor.models.keys())

In [101]:


for category in ls_categories: 
    df_output = pd.DataFrame()
    for ratio in ls_ratio: 
        # update ratio
        df_tmp = merged_data.copy()
        df_tmp[ls_quota_features] = df_tmp[ls_quota_features] * ratio
        new_featured_data = predictor.create_features(df_tmp)

        new_featured_data = new_featured_data[new_featured_data['vehicle_class'] == category]
        y_true = new_featured_data['premium']
        
        predicted_price = predictor.predict_category(category, new_featured_data[predictor.feature_names])
        
        df_tmp = ((predicted_price - y_true).abs()/y_true).describe()[['mean', 'std']]
        df_tmp = df_tmp.to_frame()
        df_tmp = df_tmp.applymap(lambda x: f"{x:.2%}" if isinstance(x, (int, float)) else x)
        df_tmp.columns = [ratio]

        df_output = pd.concat([df_output, df_tmp], axis=1)

    print(category)
    print(df_output)

Category A
       0.95   1.05   0.90   1.10   0.85   1.15   0.80   1.20
mean  2.12%  2.17%  2.20%  2.40%  2.22%  2.68%  2.25%  2.82%
std   4.91%  4.90%  4.91%  4.88%  4.90%  4.88%  4.90%  4.90%
Category B
       0.95   1.05   0.90   1.10   0.85   1.15   0.80   1.20
mean  4.44%  4.46%  4.51%  4.53%  4.58%  4.57%  4.63%  4.60%
std   8.67%  8.66%  8.63%  8.64%  8.61%  8.61%  8.57%  8.60%
Category C
       0.95   1.05   0.90   1.10   0.85   1.15   0.80   1.20
mean  1.72%  1.67%  1.79%  1.74%  1.83%  1.77%  1.94%  1.81%
std   3.63%  3.60%  3.62%  3.56%  3.61%  3.51%  3.67%  3.57%
Category D
       0.95   1.05   0.90   1.10   0.85   1.15   0.80   1.20
mean  3.08%  3.08%  3.27%  3.28%  3.42%  3.44%  3.53%  3.56%
std   7.70%  7.77%  7.66%  7.72%  7.65%  7.67%  7.67%  7.64%
Category E
       0.95   1.05   0.90   1.10   0.85   1.15   0.80   1.20
mean  4.52%  4.52%  4.64%  4.74%  4.71%  4.91%  4.79%  5.05%
std   9.02%  8.92%  8.98%  8.82%  8.95%  8.72%  8.92%  8.64%
